In [25]:
import pandas as pd
import os
import ast
import pyflakes

# Constants
QUESTION_TYPE = 1
ANSWER_TYPE = 2

file = os.path.join('..', 'data', 'pandas-processedcode-dataset.csv')

dataset = pd.read_csv(file, encoding='ISO-8859-1', error_bad_lines=False, sep=";")
print('Amount of entries in the dataset = %d' % len(dataset))
dataset


Amount of entries in the dataset = 3205


,Unnamed: 0,Unnamed: 0.1,AnswerCount,CommentCount,Id,ParentId,PostTypeId,Score,Tags,Title,Body,ViewCount,Code,AllCode,ProcessedCode
0,0,0,$2.00,7,7776679,$nan,1,25,<python><pandas>,append two data frame with pandas,<p>When I try to merge two dataframes by rows ...,"$25,479.00","['bigdata = data1.append(data2)\n', 'Exception...","['bigdata = data1.append(data2)\n', 'Exception...","['bigdata = data1.append(data2)\n', 'Exception..."
1,1,1,$7.00,3,7837722,$nan,1,187,<python><performance><for-loop><pandas>,What is the most efficient way to loop through...,<p>I want to perform my own complex operations...,"$195,364.00","['Date,Open,High,Low,Close,Volume,Adj Close\n2...","['Date,Open,High,Low,Close,Volume,Adj Close\n2...","['Date,Open,High,Low,Close,Volume,Adj Close\n2..."
2,2,2,$2.00,0,8916302,$nan,1,26,<python><csv><numpy><tab-delimited><pandas>,selecting across multiple columns with python ...,<p>I have a dataframe <code>df</code> in panda...,"$19,805.00","['df_greater_than10 = df[df[""colA""] > 10]\n']","['df', 'pandas.read_table', 'colA', 'df_greate...","['df_greater_than10 = df[df[""colA""] > 10]\n']"
3,3,3,$3.00,16,8991709,$nan,1,134,<python><r><join><data.table><pandas>,Why are pandas merges in python faster than da...,"<p>I recently came across the <a href=""http://...","$16,173.00",[],"['pandas', 'data.table', 'data.table', 'merge(...",[]
4,4,4,$8.00,5,9555635,$nan,1,19,<python><numpy><scipy><enthought><pandas>,Open source Enthought Python alternative,<p>I used Enthought's python distribution as a...,"$6,585.00",[],"['numpy', 'scipy', 'sci-kits']",[]
5,5,5,$2.00,2,9588331,$nan,1,21,<python><pandas>,Simple cross-tabulation in pandas,"<p>I stumbled across <a href=""http://pandas.py...","$9,493.00","['AB,100.00\nAB,200.00\nAC,150.00\nAD,500.00\n...","['pivot/crosstab/indexing', 'Panel', 'DataFram...","['AB,100.00\nAB,200.00\nAC,150.00\nAD,500.00\n..."
6,6,6,$3.00,0,9652832,$nan,1,38,<python><pandas><tsv>,How to I load a tsv file into a Pandas DataFrame?,<p>I'm new to python and pandas. I'm trying t...,"$32,040.00",['>>> df1 = DataFrame(csv.reader(open(\'c:/~/t...,"['tsv', 'DataFrame', '>>> df1 = DataFrame(csv....",['>>> df1 = DataFrame(csv.reader(open(\'c:/~/t...
7,7,7,$6.00,0,9758450,$nan,1,34,<python><pandas>,Pandas convert dataframe to array of tuples,<p>I have manipulated some data using pandas a...,"$36,433.00",['In [182]: data_set\nOut[182]: \n index data...,['In [182]: data_set\nOut[182]: \n index data...,"['data_set\n', '[(datetime.date(2012,2,17),24...."
8,8,8,$2.00,0,9762935,$nan,1,11,<python><pandas>,Add indexed column to DataFrame with pandas,"<p>I'm a beginning pandas user, and after stud...","$8,189.00",['df\n\n A ...,['df\n\n A ...,['df\n\n A ...
9,9,9,$2.00,0,9787853,$nan,1,17,<python><pandas>,join or merge with overwrite in pandas,<p>I want to perform a join/merge/append opera...,"$6,054.00",[],"['df1', 'df2', 'df2', 'df2', 'df1', 'df1', 'df2']",[]


In [17]:

import sys
sys.version

'3.5.2 (default, Nov 23 2017, 16:37:01) \n[GCC 5.4.0 20160609]'

In [18]:
# answers = answers[:50]
blackList = [22084742, 45695288, 45885112]

def filterBlackList(answers):
#     print('Before %d' % len(answers))
    answers = answers[~answers.Id.isin(blackList)]
#     print('Filtered %d' % len(answers))
    return answers

In [19]:
questions = dataset[dataset.PostTypeId == QUESTION_TYPE]

def reportCompilability(post, col):
    strcodes = post[col]
    codes = ast.literal_eval(strcodes)
    parseableCount = 0
    total = len(codes)
    errors = []
    errors_type = []
    for code in codes:
        # Preprocess
        #  code = preprocessCode(code)
        # Compile
        compiled, error, errtype = compileCode(code)
        # Update the report
        errors.append(str(error))
        errors_type.append(errtype)
        if compiled:
            parseableCount += 1
    return parseableCount, total, errors, errors_type


def compileCode(code):
    try:
        print('Code to compile\n' + code)
        tree = ast.parse(code)
        exec(compile(tree, filename="<ast>", mode="exec"), {})
        return True, "Sucess", "Sucess"
    except Exception as e:
        return False, e, e.__class__.__name__
    
    
def compileQuestion(question, col):
    # Get Answers
    answers = dataset[dataset.ParentId == question.Id]
    answers = filterBlackList(answers)
    
    print('----------------------------------------')
    print(question.Id)
    
    count = 0
    total = 0
    question_errors = []
    question_errors_type = []
    for _, answer in answers.iterrows():
        answerCompiled, answerTotal, errors, errors_type = reportCompilability(answer, col)
        print('Checking the answer Id = %s | Errors = %s' % (answer.Id, errors))
        count += answerCompiled
        total += answerTotal
        question_errors.extend(errors)
        question_errors_type.extend(errors_type)
        
    question['Compiled'] = count
    question['Total'] = total
    question['Errors'] = question_errors
    question['Errors Type'] = question_errors_type
    print('----------------------------------------')
    return question
    
analyzedQuestions = questions.apply(compileQuestion, axis=1, args=('ProcessedCode',)) 


----------------------------------------
7776679
Checking the answer Id = 7779260 | Errors = []
----------------------------------------
----------------------------------------
7776679
Checking the answer Id = 7779260 | Errors = []
----------------------------------------
----------------------------------------
7837722
Code to compile
pct_change = close[1:]/close[:-1]

Code to compile
pct_change = []
for row in close:
    pct_change.append(...)

Checking the answer Id = 7837947 | Errors = ["name 'close' is not defined", "name 'close' is not defined"]
Code to compile
for date, row in df.T.iteritems():
   # do some logic here

Code to compile
def my_algo(ndarray[object] dates, ndarray[float64_t] open,
            ndarray[float64_t] low, ndarray[float64_t] high,
            ndarray[float64_t] close, ndarray[float64_t] volume):
    cdef:
        Py_ssize_t i, n
        float64_t foo
    n = len(dates)

    for i from 0 <= i < n:
        foo = close[i] - open[i] # will be extremely fast




Checking the answer Id = 32307259 | Errors = ["Missing parentheses in call to 'print' (<unknown>, line 5)", 'invalid syntax (<unknown>, line 1)']
----------------------------------------
----------------------------------------
10376647
Checking the answer Id = 10377863 | Errors = []
----------------------------------------
----------------------------------------
10457584
Code to compile
In : col = ['a','b','c']

In : data = DataFrame([[1,2,3],[10,11,12],[20,21,22]],columns=col)

In : data
Out:
    a   b   c
0   1   2   3
1  10  11  12
2  20  21  22

In : data2 = data.set_index('a')

In : data2
Out:
     b   c
a
1    2   3
10  11  12
20  21  22

Checking the answer Id = 10458386 | Errors = ['invalid syntax (<unknown>, line 1)']
----------------------------------------
----------------------------------------
10464738
Code to compile
In : df = pandas.DataFrame(numpy.random.randn(5,3), index=['a','c','d','e','g'])

In : df
Out:
          0         1         2
a -1.987879 -2.028572  0.0

----------------------------------------
11073609
Code to compile
df.groupby(df.index.map(lambda t: t.minute))

Code to compile
df.groupby([df.index.map(lambda t: t.minute), 'Source'])

Code to compile
df.groupby([df['Source'],pd.TimeGrouper(freq='Min')])

Checking the answer Id = 11073962 | Errors = ["name 'df' is not defined", "name 'df' is not defined", "name 'df' is not defined"]
----------------------------------------
----------------------------------------
11077023
Checking the answer Id = 11077060 | Errors = []
Checking the answer Id = 11077215 | Errors = []
----------------------------------------
----------------------------------------
11106823
Code to compile
(x.reindex_like(y).fillna(0) + y.fillna(0)).fillna(0)

Code to compile
>>> x
   A   B   C
0  1   2 NaN
1  3 NaN   4
>>> y
    A   B   C
0   8 NaN  88
1   2 NaN   5
2  10  11  12
>>> (x.reindex_like(y).fillna(0) + y.fillna(0)).fillna(0)
    A   B   C
0   9   2  88
1   5   0   9
2  10  11  12

Checking the answer Id = 1

----------------------------------------
----------------------------------------
11622652
Checking the answer Id = 11622769 | Errors = []
Code to compile
from pandas import *

tp = read_csv('large_dataset.csv', iterator=True, chunksize=1000)  # gives TextFileReader, which is iterable with chunks of 1000 rows.
df = concat(tp, ignore_index=True)  # df is DataFrame. If errors, do `list(tp)` instead of `tp`

Checking the answer Id = 12193309 | Errors = ["File b'large_dataset.csv' does not exist"]
Code to compile
import pandas as pd
import sqlite3
from pandas.io import sql
import subprocess

# In and output file paths
in_csv = '../data/my_large.csv'
out_sqlite = '../data/my.sqlite'

table_name = 'my_table' # name for the SQLite database table
chunksize = 100000 # number of lines to process at each iteration

# columns that should be read from the CSV file
columns = ['molecule_id','charge','db','drugsnow','hba','hbd','loc','nrb','smiles']

# Get number of lines in the CSV file
nlines = subp

Code to compile
df2 = df.head(10)

Checking the answer Id = 12022003 | Errors = ["name 'df' is not defined"]
Checking the answer Id = 12025395 | Errors = []
Checking the answer Id = 12342180 | Errors = []
----------------------------------------
----------------------------------------
12047193
Code to compile
import decimal

import pydobc
import numpy as np
import pandas

cnn, cur = myConnectToDBfunction()
cmd = "SELECT * FROM myTable"
cur.execute(cmd)
dataframe = __processCursor(cur, dataframe=True)

def __processCursor(cur, dataframe=False, index=None):
    '''
    Processes a database cursor with data on it into either
    a structured numpy array or a pandas dataframe.

    input:
    cur - a pyodbc cursor that has just received data
    dataframe - bool. if false, a numpy record array is returned
                if true, return a pandas dataframe
    index - list of column(s) to use as index in a pandas dataframe
    '''
    datatypes = []
    colinfo = cur.description
    for co

...

----------------------------------------
----------------------------------------
33086881
Code to compile
df1.merge(df2,how='left', left_on='Column1', right_on='ColumnA')

Checking the answer Id = 33086953 | Errors = ["name 'df1' is not defined"]
----------------------------------------
----------------------------------------
33088010
Code to compile
df_c = pd.concat([df_a.reset_index(drop=True), df_b], axis=1)

Checking the answer Id = 33088410 | Errors = ["name 'pd' is not defined"]
----------------------------------------
----------------------------------------
33109107
Code to compile
>>> x = pandas.Series(np.random.randn(10))
>>> stats.skew(x)
-0.17644348972413657
>>> x.skew()
-0.20923623968879457
>>> stats.skew(x, bias=False)
-0.2092362396887948
>>> stats.kurtosis(x)
0.6362620964462327
>>> x.kurtosis()
2.0891062062174464
>>> stats.kurtosis(x, bias=False)
2.089106206217446

Checking the answer Id = 33109272 | Errors = ['invalid syntax (<unknown>, line 1)']
---------------------

Code to compile
# for Python 2
df.index = df.index.map(unicode) 

# for Python 3 (the unicode type does not exist and is replaced by str)
df.index = df.index.map(str)

Checking the answer Id = 35368792 | Errors = ["name 'df' is not defined"]
----------------------------------------
----------------------------------------
35414625
Code to compile
>>> df.groupby(['year', 'month', 'item'])['value'].sum().unstack('item')
item        item 1  item 2
year month                
2004 1          33     250
     2          44     224
     3          41     268
     4          29     232
     5          57     252
     6          61     255
     7          28     254
     8          15     229
     9          29     258
     10         49     207
     11         36     254
     12         23     209

Code to compile
>>> df.pivot_table(values='value', index=['year', 'month'], columns='item')
item        item 1  item 2
year month                
2004 1          33     250
     2          44     224

38154997
----------------------------------------
----------------------------------------
38231591
Code to compile
In [2]: df = pd.DataFrame({'a':[1,2,3], 'b':[{'c':1}, {'d':3}, {'c':5, 'd':6}]})

In [3]: df
Out[3]:
   a                   b
0  1           {u'c': 1}
1  2           {u'd': 3}
2  3  {u'c': 5, u'd': 6}

In [4]: df['b'].apply(pd.Series)
Out[4]:
     c    d
0  1.0  NaN
1  NaN  3.0
2  5.0  6.0

Code to compile
In [7]: pd.concat([df.drop(['b'], axis=1), df['b'].apply(pd.Series)], axis=1)
Out[7]:
   a    c    d
0  1  1.0  NaN
1  2  NaN  3.0
2  3  5.0  6.0

Code to compile
In [15]: pd.concat([df.drop('b', axis=1), pd.DataFrame(df['b'].tolist())], axis=1)
Out[15]:
   a    c    d
0  1  1.0  NaN
1  2  NaN  3.0
2  3  5.0  6.0

Checking the answer Id = 38231651 | Errors = ['invalid syntax (<unknown>, line 1)', 'invalid syntax (<unknown>, line 1)', 'invalid syntax (<unknown>, line 1)']
----------------------------------------
----------------------------------------
38250710
Code to c

Code to compile
Date      A       B       C      D      E      F      G
0   2008-03-18  24.68  164.93  114.73  26.27  19.21  28.87  63.44
1   2008-03-19  24.18  164.89  114.75  26.22  19.07  27.76  59.98
2   2008-03-20  23.99  164.63  115.04  25.78  19.01  27.04  59.61
3   2008-03-25  24.14  163.92  114.85  27.41  19.61  27.84  59.41
4   2008-03-26  24.44  163.45  114.84  26.86  19.53  28.02  60.09

Code to compile
# Put your inputs into a single list
df['single_input_vector'] = df[input_cols].apply(tuple, axis=1).apply(list)
# Double-encapsulate list so that you can sum it in the next step and keep time steps as separate elements
df['single_input_vector'] = df.single_input_vector.apply(lambda x: [list(x)])
# Use .cumsum() to include previous row vectors in the current row list of vectors
df['cumulative_input_vectors'] = df.single_input_vector.cumsum()

Code to compile
# If your output is multi-dimensional, you need to capture those dimensions in one object
# If your output is a single

Checking the answer Id = 45741989 | Errors = ['Sucess', 'invalid syntax (<unknown>, line 1)', 'invalid syntax (<unknown>, line 1)']
----------------------------------------
----------------------------------------
45820242
Code to compile
cum_r = (1 + r).cumprod()
result = cum_r * v0
for date in r.index[r.index.is_quarter_end]:
     result[date:] -= cum_r[date:] * (dist / cum_r.loc[date])

Code to compile
cum_r = (1 + r).cumprod()
t = (r.index.is_quarter_end / cum_r).cumsum()
result = cum_r * (v0 - dist * t)

Checking the answer Id = 45871837 | Errors = ["name 'r' is not defined", "name 'r' is not defined"]
----------------------------------------


In [20]:
# pd.set_option('display.max_colwidth', -1)
# analyzedQuestions[['Id', 'Title', 'Code', 'AllCode', 'Compiled','Total',  'Errors']]
10

10

In [21]:
analyzedQuestions['Compiled'].sum()
analyzedQuestions['Total'].sum()


186

3659

In [26]:
def report_errors(analyzedQuestions):
    
    errorslist = []

    for err in analyzedQuestions['Errors Type']:
        errorslist.extend(err)

    df = pd.DataFrame(errorslist)   
    print(df[0].value_counts())
    
report_errors(analyzedQuestions)

SyntaxError           1741
NameError             1301
IndentationError       257
Sucess                 186
ImportError            134
FileNotFoundError       15
AttributeError           8
TypeError                6
FutureWarning            3
DeprecationWarning       2
ValueError               2
ParserError              1
CalledProcessError       1
ShimWarning              1
OptionError              1
Name: 0, dtype: int64


In [ ]:

def addDependencies(post)

